In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt


In [0]:
transform_train = transforms.Compose([
                                transforms.RandomCrop(size=(32,32),padding_mode='edge'),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                ])
transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [66]:
train_set = torchvision.datasets.CIFAR10(root='./',train=True,download=True,transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./',train=False,download=True,transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [67]:
train_set.data.shape

(50000, 32, 32, 3)

In [0]:
train_set, val_set = torch.utils.data.random_split(train_set,[int(len(train_set)*0.9),int(len(train_set)*0.1)])

In [0]:
batch_size = 128 

train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True,drop_last=True)
val_loader =torch.utils.data.DataLoader(val_set,batch_size = batch_size,shuffle=True,drop_last = True)
test_loader =torch.utils.data.DataLoader(test_set,batch_size=batch_size,drop_last=True)

In [83]:
iter(train_loader).next()[0].shape # Check the shape of loaded data

torch.Size([128, 3, 32, 32])

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn_layer = nn.Sequential(nn.Conv2d(3,32,3,1,1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(),
                                   nn.Conv2d(32,32,3,1,1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),
                                   nn.Dropout2d(0.3),
                                   nn.Conv2d(32,64,3,1,1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.Conv2d(64,64,3,1,1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),
                                   nn.Conv2d(64,128,3,1,1),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.Conv2d(128,128,3,1,1),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.Dropout2d(0.4)
                                   )
        self.dnn_layer = nn.Sequential(nn.Linear(128*8*8,512),
                                       nn.ReLU(),
                                       nn.Linear(512,10),)
                                       
        
    def forward(self,x):
        out = self.cnn_layer(x)
        out = out.view(-1,128*8*8)
        out = self.dnn_layer(out)
        return out

In [99]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),weight_decay=0.0001,)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=20,)

cuda:0


In [100]:
loss_arr =[]
val_loss_arr=[]
best_val_acc = 0
epochs = 200
patience_epoch = 10
for i in range(epochs):
    model.train()
    correct = 0
    total = 0
    for j,[img,label] in enumerate(train_loader):
        x = img.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        _,output_index = torch.max(output,1)
        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        total += y.size()[0]
        correct += (output_index == y).sum().float()
        loss_arr.append(loss)
    
    lr_scheduler.step()    
    with torch.no_grad():
        val_correct = 0
        val_total = 0
        for k,[val_img,val_label] in enumerate(val_loader):
            val_x = val_img.to(device)
            val_y = val_label.to(device)
            val_output = model.forward(val_x)
            _,val_output_index = torch.max(val_output,1)
            val_loss = loss_func(val_output,val_y)
            val_loss_arr.append(val_loss)
            val_total += val_y.size()[0]
            val_correct += (val_output_index==val_y).sum().float()
    if best_val_acc <= 100*val_correct/val_total:
        best_val_acc = 100*val_correct/val_total
        patience = patience_epoch
    elif patience == 0:
        print('No improvement on val_acc : Stop training')
        break
    else:
        patience -= 1
    print('EPOCHS {:>3d}/{} | train_loss : {:.4f} | val_acc : {:.4f}'\
            .format(i+1,epochs,loss,100*val_correct/val_total))  
print('Best Validation Accuracy : {:.4f}'.format(best_val_acc))

EPOCHS   1/200 | train_loss : 1.0736 | val_acc : 57.2716
EPOCHS   2/200 | train_loss : 0.9802 | val_acc : 64.6635
EPOCHS   3/200 | train_loss : 0.7983 | val_acc : 71.3341
EPOCHS   4/200 | train_loss : 0.5995 | val_acc : 71.9151
EPOCHS   5/200 | train_loss : 0.6238 | val_acc : 74.6194
EPOCHS   6/200 | train_loss : 0.7360 | val_acc : 75.0000
EPOCHS   7/200 | train_loss : 0.6441 | val_acc : 76.3822
EPOCHS   8/200 | train_loss : 0.5597 | val_acc : 77.4239
EPOCHS   9/200 | train_loss : 0.5288 | val_acc : 78.3053
EPOCHS  10/200 | train_loss : 0.4242 | val_acc : 80.1082
EPOCHS  11/200 | train_loss : 0.5956 | val_acc : 80.0080
EPOCHS  12/200 | train_loss : 0.4918 | val_acc : 80.3285
EPOCHS  13/200 | train_loss : 0.3710 | val_acc : 80.5288
EPOCHS  14/200 | train_loss : 0.5380 | val_acc : 80.0080
EPOCHS  15/200 | train_loss : 0.3096 | val_acc : 80.5489
EPOCHS  16/200 | train_loss : 0.6202 | val_acc : 80.2484
EPOCHS  17/200 | train_loss : 0.3325 | val_acc : 80.6691
EPOCHS  18/200 | train_loss : 0

In [101]:
correct = 0
total = 0
with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y = label.to(device)

        output = model.forward(x)
        _,output_index = torch.max(output,1)
        total += label.size(0)
        correct += (output_index == y).sum().float()

    print('Accuracy of Testset : {:.4f}'.format(100*correct/total))

Accuracy of Testset : 83.3133
